# Demonstration notebook

This notebook is a quick demonstration of the main features and primitives of SystemFlow which you can use to investigate component and system level trade-offs in scientific computing systems. This tutorial assumes basic familiarity with Python features such as dictionaries and classes.

In [1]:
import networkx as nx
import sys
from ruamel.yaml import YAML
import numpy as np
from abc import ABC, abstractmethod
from collections import namedtuple
from copy import deepcopy
from functools import reduce
from typing import Callable, Any
from itertools import accumulate

The classes and objects we'll use to build a model of a system are defined in *Systemflow.node*:

In [2]:
from systemflow.node import *

## Introduction
In the SystemFlow framework, information originates at sources such as sensors. Sensors (and other system components) can be managed by a set of *parameters*. These can change on long time-scales, such as by replacing or upgrading equipment, but stay constant once the system is built.

For instance, let's consider a simple system: we have a 2D image sensor which acquires images at a fixed rate and carries out a convolutional filter on these images to detect certain features of interest. We may wish to consider relationships such as the number of convolutional filters used, the impact on power, and effectiveness of the system at detecting features. Over time, we may expect the resolution of the sensor to go up, and we can predict the associated demand on power and classification performance to change as well. 

## Messages
Let's start by looking at how information is transmitted through the system via *messages*. A *message* contains *fields* which represent sample-varying data, such as:
* Images
* Image features
* Classification decisions

A message also contains *properties*, which do not vary by sample, such as:
* Formats & encoding

Let's look at an example:

In [3]:
image = Message(# Fields 
                {"Image data (B)": 24e6}, # 24 MB of image data
                # Parameters
                {"Image resolution (n,n,n)": (4000, 6000, 8), # 4k x 6k pixels captured at 8 bits each
                 "Sample rate (Hz)": 1000})  #1000 frames per second

In [4]:
image.fields

{'Image data (B)': 24000000.0}

In [5]:
image.properties

{'Image resolution (n,n,n)': (4000, 6000, 8), 'Sample rate (Hz)': 1000}

## Mutations
Once originated, messages' information can be *mutated* by transforms which add additional fields and properties. Let's look at one example, a Convolve operation which uses weighted kernels to extract features such as oriented edges:

In [6]:
class Convolve(Mutate):
    """
    Models the operations and resources used during a convolution operation.
    """
    def __init__(self, name: str = "Convolve"):
        #Input message fields
        msg_fields = VarCollection()
    
        #Input message properties
        msg_properties = VarCollection(resolution = "resolution (n,n)",
                                       sample_rate = "sample rate (Hz)",)

        #Input host parameters
        host_parameters = VarCollection(kernel = "kernel (n,n)",
                                        filters = "filters (n)",)
        
        inputs = MutationInputs(msg_fields, msg_properties, host_parameters)

        #Output message fields
        msg_fields = VarCollection(features = "features (B)",)

        #Output message properties
        msg_properties = VarCollection()

        #Output host properties
        host_properties = VarCollection(ops = "conv ops (n)",)

        outputs = MutationOutputs(msg_fields, msg_fields, host_properties)

        super().__init__(name, inputs, outputs)


    def transform(self, message: Message, component: 'Component'):
        """Previously, we defined the inputs and outputs which this mutation has. Now, we create concrete
           transforms which take those inputs and set the outputs"""
        #access the required fields/properties/parameters
        res = message.properties[self.inputs.msg_properties.resolution]
        kernel_x = component.parameters[self.inputs.host_parameters.kernel][0]
        kernel_y = component.parameters[self.inputs.host_parameters.kernel][1]
        filters = component.parameters[self.inputs.host_parameters.filters]
        rate = message.properties[self.inputs.msg_properties.sample_rate]

        #calculate the number of ops required for the kernel
        kernel_ops = kernel_x * kernel_y * filters
        steps_x = (res[0] - kernel_x) // kernel_x
        steps_y = (res[1] - kernel_y) // kernel_y
        kernel_repeats = steps_x * steps_y

        #calculate the number of ops required for the kernel
        transform_operations = kernel_ops * kernel_repeats * rate

        """Above, we access the properties from the input message and host component required by the mutation,
           and calculate the outputs. These are stored in dictionaries and accessed by the host component to update
           its own properties and output message:"""
        msg_fields = {self.outputs.msg_fields.features: np.prod((steps_x, steps_y, filters)),}
        msg_properties = {}
        component_properties = {self.outputs.host_properties.ops: transform_operations,}

        return msg_fields, msg_properties, component_properties

For each mutation, the process is the same: the input message fields, properties, and the parameters controlling the mutation are declared. The informational outputs and properties of this "mutating" transform are declared, and can change the outgoing message or impart properties on the host component executing this mutation.

We've mentioned the host component a few times now, so let's look at this next.

## Components

A component is a piece of hardware which executes an algorithm implementing a mutation. In the case of our convolve operation, this could be a CPU, GPU, an FPGA, or custom chip (ASIC). 

Components can host one or more mutations which are executed sequentially. A component such as a CPU can apply multiple functions, such as *threshold*, *crop*, and *convolve* - but for now, we start with an image_sensor hosting *CollectImage*, which produces a message holding image data and associated properties. 

For each mutation, we need to know which parameters must be set on the host. We can see which parameters are required by a sequence of mutations by using *collect_parameters.* This returns a *VarCollection* that holds as class members each parameter required by the mutations. This allows you to skip copying and pasting string definitions all over the place!

In [7]:
vc_img = collect_parameters([CollectImage()])

In [8]:
vc_img

VarCollection(
    resolution='resolution (n,n)',
    bitdepth='bit depth (n)',
    readout='readout latency (s)',
    pixelenergy='pixel energy (J)',
    sample_rate='sample rate (Hz)'
)

Knowing the set of parameters we need, we instantiate them as part of the hosting component:

In [9]:
image_sensor = Component("Image sensor",
                    [CollectImage(),],
                    parameters = {vc_img.resolution: (4000, 6000),
                     vc_img.bitdepth: 16,
                     vc_img.readout: 1e-3,
                     vc_img.pixelenergy: 1e-6,
                     vc_img.sample_rate: 1000})

So, we've defined our first component - now, how do we link it to another component such as a CPU hosting image processing algorithms? The definition for the second component follows a similar format:

In [10]:
msg0 = Message({}, {})

In [11]:
CollectImage()(msg0, image_sensor)

(Message(fields={'image data (B)': np.float64(48000000.0), 'readout latency (s)': 0.001}, properties={'resolution (n,n)': (4000, 6000), 'bitdepth (n)': 16, 'sample rate (Hz)': 1000}),
 {'sensor power (W)': np.float64(24.0)})

In [12]:
cpu_mutations = [Convolve(),
                GaussianClassify(),
                DataRate(),
                ClassifiedStorageRate(),]

In [13]:
vc_cpu = collect_parameters(cpu_mutations)

In [14]:
vc_cpu

VarCollection(
    kernel='kernel (n,n)',
    filters='filters (n)',
    skill='skill (1)',
    variance='variance (1)',
    reduction='reduction (%)'
)

In [15]:
cpu = Component("CPU",
                cpu_mutations,
                parameters = {vc_cpu.kernel: (5, 5), #this kernel is 5x5 pixels...
                    vc_cpu.filters: 8, #with 8 filters
                    vc_cpu.skill: 3, # these features inform a classifier with a separation between null and positive distributions of 3.0
                    vc_cpu.variance: 1, # and each distribution has a variance of 1.0
                    vc_cpu.reduction: 0.9}) # and this classifier aims to discard 90% of data from the input distribution

## Links & ExecutionGraphs

To connect these nodes (the image sensor and CPU) and inspect their trends as parameters are varied, we'll construct an *ExecutionGraph.* An ExecutionGraph contains all the nodes which implement a directed data processing pipeline.

In [16]:
#First, we define the nodes
nodes = [image_sensor, cpu]

We've defined the nodes, but we also need to connect them via *Links.* Links transport data from one node to another, and can model the effects of this process via a single *Transport* function. Similar to mutations, *Transport* functions offer a template to describe the effect of moving data across the link by introduction new message fields, properties, and host properties: for instance, error rates, correction algorithms, and power can be modeled using the Transport function. For this example, we will use the *DefaultLink* which simply transports messages from one node to another without modifying the messages through a Transport function.

In [17]:
links = [DefaultLink("Sensor -> CPU", # A name for the link
                    nodes[0].name, # The sending (TX) node
                    nodes[1].name),] # The receiving (RX) node

Having defined our nodes and links, an ExecutionGraph can be instantiated:

In [18]:
simple_graph = ExecutionGraph("Sense/Convolve/Classify", nodes, links)

When you call an ExecutionGraph, it recursively handles passing messages from node to node, originating blank messages at each "leaf" node and propagating them upwards to the "root" node:

In [19]:
graph_2 = simple_graph()

This returns a new graph, with new properties such as "output message" stored at each node and the output properties from each mutation instantiated at each component. We can inspect the final component by inspecting the *root_node*:

In [20]:
graph_2.root_node.output_msg.fields

{'image data (B)': np.float64(48000000.0),
 'readout latency (s)': 0.001,
 'features (B)': np.int64(7664008),
 'contingency (2x2)': array([[882,  17],
        [ 17,  82]]),
 'total data (B)': np.float64(55664008.0)}

In [21]:
graph_2.root_node.output_msg.properties

{'resolution (n,n)': (4000, 6000),
 'sample rate (Hz)': 1000,
 'bitdepth (n)': 16,
 'error matrix (2p, 2p)': array([[0.98055646, 0.17499203],
        [0.01944354, 0.82500797]])}

And we can look at other nodes, messages, and properties by looking up nodes in the graph by name:

In [22]:
graph_2.get_node(image_sensor.name).properties

{'sensor power (W)': np.float64(24.0)}

Each ExecutionGraph is instantiated at iteration 1, with each successive call increasing that number.

In [23]:
graph_2.iteration

1

Interlinking multiple ExecutionGraphs which affect one another allows us to simulate feedback, with the output from one graph influencing another which loops back to the first. We'll look at this functionality in more complex examples with steering. 

For now, let's move onto a slightly more complex example with two inputs. This will allow us to demonstrate some further functionality necessary for more interesting setups. We'll create a new ExecutionGraph which hosts an additional component, a thermocouple measuring the temperature of the sample being imaged.

In [24]:
thermocouple_mutations = [CollectTemperature(),]

In [25]:
vc_thm = collect_parameters(thermocouple_mutations)

In [26]:
vc_thm

VarCollection(
    bitdepth='temperature bitdepth (n)',
    sample_rate='sample rate (Hz)',
    sensor_power='thermocouple power (W)'
)

Here, we can see that the variable collection for a thermocouple includes a sampling rate. What happens if this sample rate is different from another in the system?

## Merges

A message can only include one value per unique field and property, therefore, one value has to be selected. This is done using *Merges* on a node. The default merge behavior simply selects the first value out of any which are in conflict. However, this might not lead to the desired behavior, and so the user is warned.

In [27]:
msg0 = Message({}, {"Sample rate (Hz)": 1000})
msg1 = Message({}, {"Sample rate (Hz)": 1200})

In [28]:
OverwriteMerge()([msg0, msg1])

No merge provided for Sample rate (Hz), taking first value


Message(fields={}, properties={'Sample rate (Hz)': 1000})

In the case of this theoretical experiment where we measure the associated temperature for every image, the additional temperature sample aren't needed - they may be interpolated or discarded to match each image, and thus end up matching the lower framerate of the image sensor. We can write a new merge to take the lowest sample rate following this behavior:

In [29]:
class RateMerge(Merge):
    def __init__(self):
        # A merge allows you to set a function used to reduce a collection for
        # each desired message field and property. Here, we take the minimum
        # value of the sample rate of incoming messages:
        super().__init__({}, {vc_img.sample_rate: np.min})

Now, we can attach this Merge to the component accepting both messages as inputs which merged together. In this example, this is the CPU:

In [30]:
cpu = Component("CPU",
                cpu_mutations,
                parameters = {vc_cpu.kernel: (5, 5), #this kernel is 5x5 pixels...
                    vc_cpu.filters: 8, #...with 8 filters
                    vc_cpu.skill: 3, # these features inform a classifier with a separation between null and positive distributions of 3.0
                    vc_cpu.variance: 1, # and each distribution has a variance of 1.0
                    vc_cpu.reduction: 0.9}, # and this classifier aims to discard 90% of data from the input distribution
                merge = RateMerge(),) 

So now, let's add a component hosting the thermocouple, link them, and create a new ExecutionGraph with two leaf input sensors.

In [31]:
thermocouple = Component("Thermocouple",
                    [CollectTemperature(),],
                    {vc_thm.bitdepth: 16,
                     vc_thm.sample_rate: 1200,
                     vc_thm.sensor_power: 1e-3,},)

In [32]:
nodes = [thermocouple, image_sensor, cpu]

links = [DefaultLink("Thermocouple -> CPU",
                     thermocouple.name,
                     cpu.name),
        DefaultLink("Image sensor -> CPU",
                    image_sensor.name,
                    cpu.name)]

two_input_graph = ExecutionGraph("Sense/Convolve/Classify, 2 Inputs", nodes, links)

In [33]:
output_graph = two_input_graph()

Here, we can inspect the message and check that the sample rate has been correctly set to 1000 Hz using our new Merge:

In [34]:
output_graph.root_node.output_msg.properties[vc_img.sample_rate]

np.int64(1000)

## Metrics

Looking for the value we want in each message, component, and so forth is tedious, especially if we're constructing many graphs with multiple iterations each. We can define additional functions, *Metrics*, which are stored in an ExecutionGraph and automatically 'grab' these values and attach them to the graph for easy reference.

Let's look at one example to examine how much power the convolution operation is taking given a baseline estimate of power per operation. We can examine all the propertiest produced on the hosts in a graph, and choose one for the metric to transform into the value we're interested in:

In [35]:
output_graph.get_all_node_properties()

{'CPU': {'conv ops (n)': np.int64(191600200000),
  'storage rate (B/s)': array([ 1082305.33432831, 45923251.61719932])},
 'Thermocouple': {'thermocouple power (W)': 0.001},
 'Image sensor': {'sensor power (W)': np.float64(24.0)}}

In [36]:
Convolve().outputs.host_properties

VarCollection(
    ops='conv ops (n)'
)

In [37]:
class ConvPower(Metric):
    def __init__(self):
        super().__init__("Convolution Power", 
                         [],
                         ["conv ops (n)"])
        
    def metric(self, message: Message, properties: dict):
        matches = self.graph_matches(properties)
        op_power = 1e-10
        power = matches[0] * op_power

        metrics = {"conv power": power,}
        return metrics
    

Let's modify the graph one more time, adding a metric which we'll examine as the output of an experiment where we vary parameters controlling the operations:

In [38]:
two_input_graph = ExecutionGraph("Sense/Convolve/Classify, 2 Inputs", nodes, links, metrics=[ConvPower(),])

In [39]:
output_graph = two_input_graph()

Now, we can see that the metric values we've define are automatically produced as features of the graph:

In [40]:
output_graph.metric_values

{'conv power': np.float64(19.16002)}

If we want to locate all values with a unit (such as power (W)), we can also use a regex inside the metric and reduce over the matched values:

In [41]:
class SensorPower(Metric):
    def __init__(self):
        super().__init__("Convolution Power", 
                         [],
                         [Regex(r"power \(W\)")])
        
    def metric(self, message: Message, properties: dict):
        matches = self.graph_matches(properties)
        total_power = np.sum(matches)
        metrics = {"sensor power": total_power,}
        return metrics
    

In [42]:
two_input_graph = ExecutionGraph("Sense/Convolve/Classify, 2 Inputs", nodes, links, metrics=[ConvPower(), SensorPower()])

In [43]:
output_graph = two_input_graph()

In [44]:
output_graph.metric_values

{'conv power': np.float64(19.16002), 'sensor power': np.float64(24.001)}

Finally, let's put it all together to demonstrate the main purpose of SystemFlow: being able to rapidly link, vary, and examine the effects of changing parameters.

## Demonstration

Let's imagine that we've established an empirical relationship between two elements: the number of features used in the convolutional kernel, and the performance of the classifier - by adding more features, we can improve the separation of the classifier between null and positive distributions. However, this comes at the cost of increased computational power. Using the relationship we've found, let's sweep through these values and examine the power it takes to reach different levels of performance.

In [45]:
def sweep_filters(filters: int, exg: ExecutionGraph):
    # an empirical relationship we assume between the classifier skill and number of filters
    new_skill = 4 * np.log10(filters)
    new_params = {"CPU": {vc_cpu.skill: new_skill,
                          vc_cpu.filters: filters,}}
    # we can simply call an existing graph with new parameters
    new_exg = exg.with_updated_parameters(new_params)()
    # and extract the performance of the classifier
    cont = new_exg.root_node.output_msg.fields["contingency (2x2)"]
    #then convert this to an F1 score
    tn, fn = get_rejected(cont)
    fp, tp = get_passed(cont)
    f1 = (2 * tp) / (2 * tp + fp * fn)
    power = new_exg.metric_values["conv power"]
    return power, f1

In [46]:
n_filters = np.arange(1, 20)

In [47]:
data = list(map(lambda x: sweep_filters(x, two_input_graph), n_filters))

In [48]:
stacked = np.array(data)

In [49]:
import plotly.graph_objects as go

In [50]:
fig = go.Figure()

# Add Power trace
fig.add_trace(go.Scatter(
    x=n_filters,
    y=stacked[:,0],
    mode='lines',
    name='Power',
    line=dict(color='blue')
))

# Add F1 Score trace
fig.add_trace(go.Scatter(
    x=n_filters,
    y=stacked[:,1],
    mode='lines',
    name='F1 Score',
    line=dict(color='red'),
    yaxis="y2"
))

# Update layout
fig.update_layout(
    title_text='Power and F1 Score By Convolution Filters',
    xaxis_title='Filters',
    yaxis_title='Power (W)',
    yaxis2=dict(
        title="F1 Score",
        overlaying="y",
        side="right",
        range=[0, 1]
    ),
    legend_title_text='Metrics',
    width = 800,
    height = 600
)

fig